# December 17, 2024

https://adventofcode.com/2024/day/17

In [4]:
import re

In [1]:
DEBUG = False
def dprint( *args ):
    if DEBUG:
        return print(*args)


In [6]:
test_str = f'''Register A: 729
Register B: 0
Register C: 0

Program: 0,1,5,4,3,0'''
test_str = test_str.split("\n")

In [3]:
fn = "../data/2024/17.txt"
with open(fn, "r") as file:
    text = file.readlines()
puzz_str = [x.strip() for x in text]

# Part 1


In [54]:
class Computer:
    def __init__(self, text):
        self.A = int(re.search("\d+$", text[0])[0])
        self.B = int(re.search("\d+$", text[1])[0])
        self.C = int(re.search("\d+$", text[2])[0])
        self.program = [int(x) for x in text[4][9:].split(",") ]

        # points at current index in program sequence
        self.ptr = 0

        # numerical output buffer 
        self.output = list()

    def run(self):
        while self.ptr < len(self.program):
            self.execute()


        return ",".join( [str(x) for x in self.output] )
    
    def execute(self):
        op_list = ["adv", "bxl", "bst", "jnz", "bxc", "out", "bdv", "cdv"]
        opno = self.program[ self.ptr ]
        operand = self.program[ self.ptr + 1]
        self.ptr += 2
        op = getattr(self, op_list[opno])
        return op( operand )
        
    # operand interpretation
    def combo(self, n):
        if n <= 3:
            return n
        if n == 4:
            return self.A
        if n == 5:
            return self.B
        if n == 6:
            return self.C
        if n == 7:
            raise BaseException("Combo operand 7 is invald")
        
    def literal(self, n):
        return n

    # operator implementation
    def adv(self, operand):
        # op 0
        x = self.combo(operand)
        self.A = int( self.A / (2**x) )
        return self.A
    
    def bxl(self, operand):
        # op 1
        x = self.literal(operand)
        self.B ^= x
        return self.B
    
    def bst(self, operand):
        # op 2
        x = self.combo(operand)
        self.B = x % 8
        return self.B
    
    def jnz(self, operand):
        # op 3
        x = self.literal(operand)
        if self.A != 0:
            self.ptr = x
        return None
    
    def bxc(self, operand):
        # op 4
        self.B ^= self.C
        return self.B
    
    def out(self, operand):
        # op 5
        x = self.combo(operand)
        self.output.append( x % 8 )
        return None
    
    def bdv(self, operand):
        # op 6
        x = self.combo(operand)
        self.B = int( self.A / (2**x) )
        return self.B
    
    def cdv(self, operand):
        # op 7
        x = self.combo(operand)
        self.C = int( self.A / (2**x) )
        return self.C

In [55]:
test = Computer(test_str)
test.run()

'4,6,3,5,6,3,5,2,1,0'

In [56]:
Computer(puzz_str).run()

'7,0,3,1,2,6,3,7,1'

# Part 2

The trick here is to solve from right to left.

In [304]:
def part2( text ):

    compy = Computer(text)
    target = compy.program
    A = 0
    while True:
        dprint("byteprefix:", A)
        solution = solve( target, A, text )
        if solution is not None:
            break
        A += 1

    print("Found a solution starting with byte prefix", A)
    compy.A = solution
    out = compy.run()
    result = [int(x) for x in out.split(",")]
    assert compy.program == result
    return solution

def solve( target, A_so_far, text ):
    if len(target) == 0:
        return A_so_far
    
    # find possibilities to hit the next target
    nbo = next_byte_options( target[-1], A_so_far, text )

    # try each of those possibilities
    for nb in nbo:
        result = solve( target[:-1], A_so_far*8 + nb, text )
        if result is not None:
            return result
        
    # poo -- no solution found
    dprint("No solution for target", target, "starting with A", A_so_far)
    return None



def next_byte_options( target, A_so_far, text ):
    nbo = list()
    for n in range(0,8):
        comp = Computer(text)
        comp.A = A_so_far * 8 + n
        out = comp.run()
        byte = int(out[0])
        if byte == target:
            nbo.append(n)

    return nbo

In [305]:
part2(puzz_str)

byteprefix: 0
No solution for target [2, 4, 1, 5, 7, 5, 0, 3, 4, 1, 1, 6] starting with A 1538
No solution for target [2, 4, 1, 5, 7, 5, 0, 3, 4, 1, 1, 6] starting with A 1540
No solution for target [2, 4, 1, 5, 7, 5, 0, 3, 4, 1, 1, 6] starting with A 1542
No solution for target [2, 4, 1, 5, 7, 5, 0, 3, 4, 1, 1, 6] starting with A 1543
No solution for target [2, 4, 1, 5, 7, 5, 0, 3, 4, 1, 1, 6, 5] starting with A 192
No solution for target [2, 4, 1, 5, 7, 5, 0, 3, 4, 1, 1, 6] starting with A 1570
No solution for target [2, 4, 1, 5, 7, 5, 0, 3, 4, 1, 1, 6] starting with A 1572
No solution for target [2, 4, 1, 5, 7, 5, 0, 3, 4, 1, 1, 6] starting with A 1575
No solution for target [2, 4, 1, 5, 7, 5, 0, 3, 4, 1, 1, 6, 5] starting with A 196
Found a solution starting with byte prefix 0


109020013201563